In this ipynb file, we implement SDP for optimal recovery. Our implementation has following features:

1. the SDP has only one input: the QEC matrix.

2. the choi matrix is restricted on the error subspace. As a result, the choi matrix is l*d*d dimensional, where l is the number of Kraus operators taken into consideration, d=2 for qubit code.

Furthermore, we provide an near optimal solution to the SDP

In [1]:
import warnings
warnings.filterwarnings("error")
import numpy as np
import math
import mpmath
from scipy.special import hermite,  eval_genlaguerre, loggamma
from functools import lru_cache
import cvxpy as cp 
import scipy


sinh = np.sinh
cosh = np.cosh
tanh = np.tanh
sqrt = np.sqrt
pi = np.pi
exp = np.exp
conj = np.conj
ln = np.log

Construct QEC matrix. Here we use Gaussian envelope finite energy GKP code as an example, you can substitute it for other codes. The GKP matrix $M_{l,\mu,l',\mu'}$ aligns as following:

In [2]:

# generate m and M matrix for GKP()
class QECmat_GKP:
    def __init__(self,Delta,gamma,l_cut,m_sum_cutoff=20,M_sum_cutoff=5):
        self.gamma = gamma # damping rate gamma for amplitude damping channel
        self.Delta = Delta  # characterize energy
        self.n_Delta = 1/(exp(2*Delta**2)-1) # approximate average phonon number
        self.l_cut = l_cut  # how much kraus operator taking into consideration # typically 20
        self.m_sum_cutoff = m_sum_cutoff    # typically 20
        self.M_sum_cutoff = M_sum_cutoff    # typically 5
    
    # overlap of GKP code basis
    def m(self,keep_real = True):
        cutoff = self.m_sum_cutoff
        Delta = self.Delta
        cutoff = 20
        res = np.zeros((2,2),dtype=complex)
        for mu1 in [0,1]:
            for mu2 in [0,1]:
                for n1 in range(-cutoff,cutoff):
                    for n2 in range(-cutoff,cutoff):
                        Lambda = np.sqrt(np.pi/2)*(2*n1+mu1-mu2+1.j*n2)
                        res[mu1,mu2] += 1/(2*sqrt(np.pi)*(1-np.exp(-2*Delta**2)))* np.exp(1.j*np.pi*(n1+(mu1+mu2)/2)*n2)*np.exp(-1/(2*np.tanh(Delta**2))*abs(Lambda)**2) 
        # keep only real part
        if keep_real == True:
            res = res.real
        return np.matrix(res)

    # average photon number(rigorous result)
    def n_ave(self,sum_cutoff):
        Delta = self.Delta
        n_D = self.n_Delta
        factor = 0.5 / (2*sqrt(pi)*(1-exp(-2*Delta**2)))
        m0 = np.matrix([[m(Delta,mu,nu) for nu in [0,1]] for mu in [0,1]])
        cutoff = 20
        def K(Delta,mu,mup,cutoff):
            res = 0
            t2 = np.tanh(Delta**2)
            s2 = np.sinh(Delta**2)
            for n1 in range(-sum_cutoff,sum_cutoff):
                for n2 in range(-sum_cutoff,sum_cutoff):
                    Lambda = sqrt(pi/2)*(2*n1+mu-mup+n2*1.j)
                    alpha = exp(Delta**2)/sqrt(gamma+1/n_D)*conj(Lambda)
                    res += exp(-pi/(2*t2)*abs(Lambda)**2)*exp(1.j*pi*(n1+(mu+mup)/2)*n2)* abs(Lambda)**2 / 4 / s2**2
            return res
        K0 = np.matrix([[K(Delta,mu,nu,cutoff) for nu in [0,1]] for mu in [0,1]])
        return n_D - factor*np.trace(np.linalg.inv(m0)@K0)

    def _lDl(self,l,lp,alpha):
        assert type(l) == int
        assert type(lp) == int
        if alpha == 0:
            if l != lp:
                return 0
            else:
                fac = 0
        else:
            fac = (l-lp)*ln(alpha)
        if l>=lp:
            res = exp(-0.5*abs(alpha)**2 + 0.5*(loggamma(lp+1)-loggamma(l+1))+fac)*eval_genlaguerre(lp,l-lp,abs(alpha)**2)
        else:
            l,lp=lp,l
            res = self._lDl(l,lp,alpha)
            res = res.conjugate()
            l,lp=lp,l
        return res

    # error correction matrix
    # metric of basis E_l \ket{mu}, i.e. describe overlap and norm of GKP E_l|0> E_l|1> , for l = 0,1,2,...
    # return a 2l_cutoff * 2l_cutoff dimentional matrix
    def _M(self,keep_real = True):
        sum_cutoff = self.M_sum_cutoff
        l_cutoff = self.l_cut
        Delta = self.Delta
        gamma = self.gamma
        res = np.zeros((2*l_cutoff,2*l_cutoff),dtype=complex)
        # element of M matrix
        def M_ele(cutoff,l,mu,lp,mup):
            n_D = 1/(exp(2*Delta**2)-1)
            factor = (gamma*n_D)**((l+lp)/2)/(gamma*n_D+1)**((l+lp)/2+1) / (2*sqrt(pi)*(1-exp(-2*Delta**2)))
            res = 0
            t = np.tanh(Delta**2/2)
            for n1 in range(-cutoff,cutoff):
                for n2 in range(-cutoff,cutoff):
                    Lambda = sqrt(pi/2)*(2*n1+mu-mup+n2*1.j)
                    alpha = exp(Delta**2)/sqrt(gamma+1/n_D)*conj(Lambda)
                    lDl = self._lDl(l,lp,alpha)
                    res += exp(-(1-gamma)/2/(gamma+1/n_D)*abs(Lambda)**2)*exp(1.j*pi*(n1+(mu+mup)/2)*n2)*lDl
            return (factor*res).real
        for i in range(2*l_cutoff):
            for j in range(2*l_cutoff):
                res[i,j] = M_ele(sum_cutoff, i//2, i%2, j//2, j%2)
        # keep only real part
        if keep_real == True:
            res = res.real
        return res
    
    # transform into orthogonalized and normalized GKP basis
    def orth_M(self):
        Delta = self.Delta
        gamma = self.gamma
        Mmat = self._M()
        mmat = self.m()
        u, s, vh = np.linalg.svd(mmat, full_matrices=True)
        U = np.diag(np.array(s)**(-0.5))@vh
        U = np.kron(np.eye(self.l_cut), U)
        Mmat = U@Mmat@U.transpose()
        #print(U@Mmat@U.transpose())
        return Mmat


implement SDP process

In [3]:
# processing QEC matrix
# only one input: QEC matrix (with orthornormal code basis)
# output: transpose fidelity, SDP fidelity

class process_QECmat:
    def __init__(self,QECmat,dimL,d):
        self.QECmat = QECmat
        self.dimL = dimL    # number of kraus operator taking into consideration
        self.d = d # d means qudit, 2 for qubit
        self.dimQECmat = d*dimL
        assert self.QECmat.shape == (self.dimQECmat,self.dimQECmat)
        
    # compute transpose fidelity with equation 
    def transpose_infid_M(self):
        dimL = self.dimL
        #print('n_Delta',n_Delta(Delta))
        #print('n_ave',n_ave(Delta))
        #test_GKPstate_nBasis(Delta)
        #print('orth_M test',orth_M(Delta, 0, 3,5))
        Msqrt = scipy.linalg.sqrtm(self.QECmat)
        ptrMsqrt = Msqrt.reshape([dimL,2,dimL,2])
        ptrMsqrt = np.matrix([[ptrMsqrt[i,0,j,0]+ptrMsqrt[i,1,j,1] for j in range(dimL)] for i in range(dimL)])
        fid = (1/self.d**2)*np.trace(ptrMsqrt@ptrMsqrt.transpose())
        #print(1-fid)
        return 1-fid


    # find an optimized recovery with SDP
    # use phonon number basis
    def SDP_new(self,eps,verbose = False):
        dimL = self.dimL
        d = self.d
        # the SDP problem is
        # max_R Tr 1/4 + 1/4 \sum_{i=x,y,z} A_i N_\gamma (\sigma_i)
        # R = 1/2 I_n \otimes I_code + \sum_{i=x,y,z} A_i \otimes \sigma_i >= 0
        #compute N_gamma_othNor_pauli(sigma_i)

        R = cp.Variable((dimL*d*d,dimL*d*d), symmetric=True)

        # compute partial trace of R
        constraints = [R >> 0]
        for i1 in range(dimL):
            for j1 in range(dimL):
                for mu1 in range(d):
                    for nu1 in range(d):
                        cons = np.zeros([dimL*d*d,dimL*d*d])
                        for k in range(self.d):
                            cons[i1*d*d+mu1*d+k,j1*d*d+nu1*d+k] = 1
                        constraints += [cp.trace(cp.transpose(cons)@R)==self.QECmat[i1*d+mu1,j1*d+nu1]]
        matK = np.zeros([dimL*d*d,dimL*d*d])
        for i in range(dimL):
            for mu in range(d):
                for mup in range(d):
                    matK[i*d*d+mu*d+mu,i*d*d+mup*d+mup] = 1
        # solve SDP                
        prob = cp.Problem(
            (1/d**2)*cp.Maximize(cp.trace(cp.transpose(matK)@R)),
            constraints
        )

        
        # optimization---')
        prob.solve(eps=eps,verbose = verbose)
        #print('---end optimization---')
        # result choi matrix 
        #choi = 1/2*cp.kron(A0,np.identity(2)) + cp.kron(A1, sigma1)+ cp.kron(A2, sigma2)+ cp.kron(A3, sigma3) 
        choi = None
        return prob.value, choi, prob._solver_stats.num_iters
    
    
    # find an optimized recovery with SDP
    # use phonon number basis
    def SDP_new_new(self,eps,verbose = False):
        dimL = self.dimL
        d = self.d
        # the SDP problem is
        # max_R Tr 1/4 + 1/4 \sum_{i=x,y,z} A_i N_\gamma (\sigma_i)
        # R = 1/2 I_n \otimes I_code + \sum_{i=x,y,z} A_i \otimes \sigma_i >= 0
        #compute N_gamma_othNor_pauli(sigma_i)

        R = cp.Variable((dimL*d*d,dimL*d*d), symmetric=True)

        # compute partial trace of R
        constraints = [R >> 0]
        identMat = np.eye(dimL*d)
        for i1 in range(dimL):
            for j1 in range(dimL):
                for mu1 in range(d):
                    for nu1 in range(d):
                        cons = np.zeros([dimL*d*d,dimL*d*d])
                        for k in range(self.d):
                            cons[i1*d*d+mu1*d+k,j1*d*d+nu1*d+k] = 1
                        constraints += [cp.trace(cp.transpose(cons)@R)==identMat[i1*d+mu1,j1*d+nu1]]
        _Msqrt = np.matrix(scipy.linalg.sqrtm(self.QECmat))
        Msqrt = np.matrix(np.zeros([dimL*d*d,dimL*d*d]))
        for i in range(dimL*d):
            for j in range(dimL*d):
                for mu in range(d):
                    Msqrt[i*d+mu,j*d+mu] = _Msqrt[i,j]
        
        matK = np.matrix(np.zeros([dimL*d*d,dimL*d*d]))
        for i in range(dimL):
            for mu in range(d):
                for mup in range(d):
                    matK[i*d*d+mu*d+mu,i*d*d+mup*d+mup] = 1
        # solve SDP
        matK = Msqrt * matK * Msqrt                
        prob = cp.Problem(
            (1/d**2)*cp.Maximize(cp.trace(cp.transpose(matK)@R)),
            constraints
        )

        
        # optimization---')
        prob.solve(eps=eps,verbose = verbose)
        #print('---end optimization---')
        # result choi matrix 
        #choi = 1/2*cp.kron(A0,np.identity(2)) + cp.kron(A1, sigma1)+ cp.kron(A2, sigma2)+ cp.kron(A3, sigma3) 
        choi = None
        return prob.value, choi, prob._solver_stats.num_iters
    
    # generate choi matrix for transpose channel


do some test

In [4]:
for gamma in np.linspace(0,0.1,11):
    print('--- gamma =',gamma)
    Delta = 0.481
    dimL=20
    cutoff = 5
    l_cut = 20
    m_sum_cutoff=20
    M_sum_cutoff=5
    
    n_cut = 40
    i_cut = 10
    eps = 1e-6
    
    ####################
    # implement new transpose and new GKP
    ####################
    M_GKP = QECmat_GKP(Delta = Delta, gamma = gamma, l_cut = l_cut, m_sum_cutoff = m_sum_cutoff, M_sum_cutoff = M_sum_cutoff).orth_M()
    GKP_QECprocess = process_QECmat(QECmat = M_GKP,dimL = l_cut,d = 2)


    # do optimization
    new_transpose = GKP_QECprocess.transpose_infid_M()
    print('new_transpose:',new_transpose)
    #new_SDP = GKP_QECprocess.SDP_new(eps = eps)
    #print('new SDP:',1-new_SDP[0],'iter:',new_SDP[2])
    new_new_SDP = GKP_QECprocess.SDP_new_new(eps = eps)
    print('new new SDP:',1-new_new_SDP[0],'iter:',new_new_SDP[2])

--- gamma = 0.0


new_transpose: 0.0
new new SDP: -7.451015808790373e-07 iter: 75
--- gamma = 0.01
new_transpose: 0.0007032768163498515
new new SDP: 0.0006820488505310252 iter: 1150
--- gamma = 0.02
new_transpose: 0.0015788792758916204
new new SDP: 0.0014937535546506453 iter: 875
--- gamma = 0.03
new_transpose: 0.002638626255126786
new new SDP: 0.002444421399645069 iter: 1800
--- gamma = 0.04
new_transpose: 0.00389402609116718
